In [ ]:
import sympy as sp
import numpy as np
from sympy.physics.quantum import TensorProduct

In [ ]:
def get_U1Gauge(states):
    #calculate the U1_gauges with the wannier states
    
    #get the shape of the data
    (Nx, Ny, _) = states.shape

    #create container for the U1 gauges
    U1_gauges = np.zeros(Nx*Ny*2).reshape((Nx,Ny,2)).astype(np.complex128)

    #iterate through the Brillouin zone
    for ii in range(Nx):
        for jj in range(Ny):
            #calculate the overlap between neighbouring Wannier states in the x and y direction
            dotprod1 = np.dot(np.conj(states[(ii+1)%Nx,jj,:].T), states[ii,jj,:])
            dotprod2 = np.dot(np.conj(states[ii,(jj+1)%Ny,:].T), states[ii,jj,:])

            #store the U1 gauges
            U1_gauges[ii,jj,0] = dotprod1/np.abs(dotprod1)
            U1_gauges[ii,jj,1] = dotprod2/np.abs(dotprod2)

    #return the U1 gauges
    return U1_gauges

def get_numerical_BerryCurvature(states):
    #numerical calculation of the Berry curvature for the Wannier basis in a fixed kz plane

    #get the shape of the data
    (Nx, Ny, _) = states.shape

    #calculate the U1 gauges in the lattice
    U1_gauges = get_U1Gauge(states)
    
    #define container for the Berry curvature as a function of (kx,ky)
    BerryCurvature = np.zeros(Nx*Ny).reshape((Nx,Ny)).astype(np.complex128)

    #iterate through the lattice 
    for ii in range(Nx):
        for jj in range(Ny):
            #calculate the Berry curvature
            BerryCurvature[ii,jj]  = np.log(U1_gauges[ii,jj,0] 
                                            * U1_gauges[(ii+1)%Nx,jj,1] 
                                            / U1_gauges[ii,(jj+1)%Ny,0] 
                                            / U1_gauges[ii,jj,1])
            
    #return the Berry curvature
    return BerryCurvature

In [ ]:
#create the Pauli matrices
s0 = sp.eye(2)
sx = sp.Matrix([[0, 1], [1, 0]])
sy = sp.Matrix([[0, -sp.I], [sp.I, 0]])
sz = sp.Matrix([[1, 0], [0, -1]])

In [ ]:
#create parameters and the Pauli matrices 
kx_sym, ky_sym, kz_sym = sp.symbols('k_x k_y k_z', real = True)
ksymbols = [kx_sym, ky_sym, kz_sym]
alpha_sym = sp.symbols('alpha', real = True, positive = True)
gamma_z, lambda_z = sp.symbols('gamma_z lambda_z', real = True)

In [ ]:
H_dartboard = - sp.sin(kx_sym) * sp.sin(2*ky_sym) * sx
H_dartboard += sp.sin(2*kx_sym) * sp.sin(ky_sym) * sy 
H_dartboard += (alpha_sym + sp.cos(2*kx_sym) + sp.cos(2*ky_sym)) * sz

In [ ]:
H_layered = sp.Matrix(np.zeros((4,4)))
H_layered += TensorProduct(sz,H_dartboard)
H_layered += TensorProduct(sy,s0) * lambda_z * sp.sin(kz_sym)
H_layered += TensorProduct(sx,s0) * (gamma_z + lambda_z * sp.cos(kz_sym))

In [ ]:
H_layered

In [ ]:
#define system size
Nx = 30
Ny = 30
Nz = 30
Nbands = 4
Nocc = 2

params = {}
params["Nx"] = Nx
params["Ny"] = Ny
params["Nz"] = Nz
params["Nbands"] = Nbands
params["Nocc"] = Nocc

In [ ]:
Kxs = np.linspace(0, 2*np.pi, Nx, endpoint = False)
Kys = np.linspace(0, 2*np.pi, Ny, endpoint = False)
Kzs = np.linspace(0, 2*np.pi, Nz, endpoint = False)

In [ ]:
#fix the parameters of the model
H_fixparam = H_layered.subs({alpha_sym : 1.0, gamma_z: 0.5, lambda_z: 1})

In [ ]:
#calculate the eigenvalues and eigenstates of the system
hfunc = sp.lambdify((kx_sym,ky_sym,kz_sym), H_fixparam, modules = "numpy")

In [ ]:
eigenvalues = np.zeros((Nx,Ny,Nz,Nbands))
eigenstates = np.zeros((Nx,Ny,Nz,Nbands,Nbands), dtype = np.complex128)

for i in range(Nx):
    for j in range(Ny):
        for k in range(Nz):
            vals, vecs = np.linalg.eigh(hfunc(Kxs[i],Kys[j],Kzs[k]))

            ind = np.argsort(vals)

            eigenvalues[i,j,k] = vals[ind]
            eigenstates[i,j,k] = vecs[:,ind]

In [ ]:
Links_z = np.zeros((Nx,Ny,Nz,Nocc,Nocc), dtype = np.complex128)

for i in range(Nx):
    for j in range(Ny):
        for k in range(Nz): 
            ol = eigenstates[i,j,(k+1)%Nz,:,:Nocc].conj().T @ eigenstates[i,j,k,:,:Nocc]
            S, _, V = np.linalg.svd(ol)

            Links_z[i,j,k] = S @ V

Wilsonloops = np.zeros((Nx,Ny,Nz,Nocc,Nocc), dtype = np.complex128)

for i in range(Nx):
    for j in range(Ny):
        for k in range(Nz):
            W = np.eye(Nocc).astype(np.complex128)
            for kp in range(Nz):
                W = Links_z[i,j,(k+kp)%Nz] @ W

            Wilsonloops[i,j,k] = W

nuvals = np.zeros((Nx,Ny,Nz,Nocc))
nuvecs = np.zeros((Nx,Ny,Nz,Nocc, Nocc), dtype = np.complex128)

for i in range(Nx):
    for j in range(Ny):
        for k in range(Nz):
            vals, vecs = np.linalg.eig(Wilsonloops[i,j,k])
            angles = np.angle(vals) / (2 * np.pi)

            ind = np.argsort(angles)

            nuvals[i,j,k] = angles[ind]
            nuvecs[i,j,k] = vecs[:,ind]

In [ ]:
#compute Wannier basis
wannierbasis = np.zeros((Nx,Ny,Nz,Nbands,Nocc)).astype(np.complex128)

for i in range(Nx):
    for j in range(Ny):
        for k in range(Nz):
            for m in range(Nocc):
                wannierbasis[i,j,k,:,m] = eigenstates[i,j,k,:,0] * nuvecs[i,j,k,0,m] + eigenstates[i,j,k,:,1] * nuvecs[i,j,k,1,m]

In [ ]:
Nx2 = int(Nx/2)
Ny2 = int(Ny/2)

cornerBZ_states = wannierbasis[:Nx2,:Ny2,2,:,0]

In [ ]:
bc = get_numerical_BerryCurvature(cornerBZ_states)
np.sum(bc / (2*np.pi*1.j))